In [1]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

In [2]:
prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare'}`




/Users/newid/Workspace/full-stack-gpt/venv/lib/python3.10/site-packages/langchain/utilities/duckduckgo_search.py:43: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


4、在连锁零售门店中有时称单品为一个SKU（中文译为最小存货单位，英文全称为 stock keeping unit, 简称SKU，定义为保存 … 先看看stock一词各个义项在英语中的使用分部，可见 “股票” 必然是绝大多数时候的含义，而它的本意 “树干” 早就很少使用了，这中间 … 今天试着用了一下Adobe Stock，发现还可以自己上传作品售卖并获得收入，但是需要提交ID Card照片才行，而… Aug 19, 2021 · 以下是关于stock、security、fund、bond这几个金融术语的详细解释： Security（证券） 证券是金融产品的统 … Awesome Stock Video Clips, Stock Music, Sound Effects & Video Templates. All available for free! Mixkit is a free gallery …
Invoking: `CompanyOverview` with `{'symbol': 'NET'}`


{'Symbol': 'NET', 'AssetType': 'Common Stock', 'Name': 'Cloudflare Inc', 'Description': 'Cloudflare Inc. (ticker: NET) is a prominent global cloud services provider headquartered in San Francisco, California, specializing in enhancing the security, performance, and resilience of websites and applications. Through its robust suite of services, including content delivery networks, DDoS mitigation, and comprehensive internet security solutions, Cloudflare supports millions of clients across diverse sectors in optimizing their online

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 23610 tokens (23339 in the messages, 271 in the functions). Please reduce the length of the messages or functions.

In [3]:
alpha_vantage_api_key

'AK1KVGB89GWAT8AA'